# **Điểm chuẩn - Preprocessing**

Preprocessing data điểm chuẩn đã crawl về từ web [huongnghiep.hocmai.vn](https://huongnghiep.hocmai.vn/diem-chuan/)

## **Intro:**
- Bộ dữ liệu raw là các file csv như sau:  

    ```
    CRAWLER-HOCMAI  
    └── data  
        └── full  
            ├── diemchuan2018_demo.csv  
            ├── diemchuan2019_full.csv  
            ├── diemchuan2020_full.csv  
            ├── diemchuan2021_full.csv  
            ├── diemchuan2022_full.csv  
            ├── diemchuan2023_full.csv  
    ```
- Tương ứng mỗi file là dữ liệu điểm chuẩn của 1 năm.
    - 2 file `full` là dữ liệu đã được crawled đầy đủ.
    - 4 file `demo` là dữ liệu ***gần như*** đầy đủ, vì trong quá trình crawl có bị missing một vài trường. Sẽ cố gắng cập nhật đầy đủ sớm hơn.
- Cấu trúc dữ liệu mỗi file gồm 7 cột `university_code`, `university_name`, `major_code`, `major_name`, `subject_group`, `point`, `note`, `year`. 

    Một mẫu dữ liệu minh hoạ như sau: 
    
| university_code | university_name                 | major_code | major_name               | subject_group       | point | note             | year |
|-----------------|---------------------------------|------------|--------------------------|---------------------|-------|------------------|------|
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7140234    | Sư phạm Tiếng Trung Quốc | D01, D15, D04, D45  | 24.85 | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7140233    | Sư phạm Tiếng Pháp       | D01, D15, D03, D44  | 19    | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7220204    | Ngôn ngữ Trung Quốc      | D01, D15, D04, D45  | 23    | Tốt nghiệp THPT  | 2023 |
| UFL-HU          | Đại Học Ngoại Ngữ – Đại Học Huế | 7220210    | Ngôn ngữ Hàn Quốc        | D01, D14, D15       | 22.5  | Tốt nghiệp THPT  | 2023 |



## **Target**
- Kiểm tra tính đầy đủ của dữ liệu (về số trường đại học).

- Kiểm tra thiếu dữ liệu của từng dòng.

- Kiểm tra trùng lặp.

- **Phân tách dữ liệu cho từng loại điểm**


## **Start here**

### **import các thư viện**

In [39]:
import pandas as pd
import numpy as np
import os

### **Chỉ định đường dẫn thư mục**

In [40]:
# Directory containing the CSV files
directory = './data/full'


### **Đọc các file csv vào df**

In [41]:
# Dictionary to store DataFrames
dfs = {}
raw = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct full file path
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Store the DataFrame in the dictionary with the filename (without .csv) as the key
        dfs[filename.split('_')[0]] = df
        raw.append(df)

# Display the keys of the dictionary to confirm the DataFrames are loaded
dfs.keys()


dict_keys(['diemchuan2018', 'diemchuan2019', 'diemchuan2020', 'diemchuan2021', 'diemchuan2022', 'diemchuan2023'])

In [42]:
shapes = {name: df.shape for name, df in dfs.items()}
shapes

{'diemchuan2018': (4289, 8),
 'diemchuan2019': (5706, 8),
 'diemchuan2020': (4924, 8),
 'diemchuan2021': (6561, 8),
 'diemchuan2022': (7991, 8),
 'diemchuan2023': (7947, 8)}

In [43]:
full_df = pd.concat(raw, ignore_index=True)
full_df.head()

,university_code,university_name,major_code,major_name,subject_group,point,note,year
0,TVU,Đại Học Trà Vinh,7210201,Âm nhạc học,N00,15.00,NaN,2018
1,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.90,Tốt nghiệp THPT,2018
2,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90, XDHB",27.25,Học bạ,2018
3,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,DGNLHCM,22.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2018
4,VNUHCM - UIT,UIT - Đại Học Công Nghệ Thông Tin – Đại Học Qu...,7480202_CLCA,An toàn thông tin,"A00, A01, D01",20.10,chất lượng cao,2018


In [44]:
full_df.shape

(37418, 8)

In [45]:
uni_info = pd.read_csv("./data/university/university.csv")
uni_info.drop(columns='url',inplace=True)

In [46]:
full_df = pd.merge(full_df, uni_info, how="left", on=["university_code", "university_name"])
full_df

,university_code,university_name,major_code,major_name,subject_group,point,note,year,province,region,admission_code,institution_type
0,TVU,Đại Học Trà Vinh,7210201,Âm nhạc học,N00,15.00,NaN,2018,Trà Vinh,Miền Nam,DVT,Công lập
1,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.90,Tốt nghiệp THPT,2018,Hồ Chí Minh,Miền Nam,SPK,Công lập
2,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90, XDHB",27.25,Học bạ,2018,Hồ Chí Minh,Miền Nam,SPK,Công lập
3,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,DGNLHCM,22.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2018,Hồ Chí Minh,Miền Nam,SPK,Công lập
4,VNUHCM - UIT,UIT - Đại Học Công Nghệ Thông Tin – Đại Học Qu...,7480202_CLCA,An toàn thông tin,"A00, A01, D01",20.10,chất lượng cao,2018,Hồ Chí Minh,Miền Nam,QSC,Công lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37413,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập
37414,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập
37415,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập
37416,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,Công lập


#### Đổi tên các cột lại thành tiếng Việt

In [47]:
# Dictionary to map old column names to new column names
column_mapping = {
    'province': 'Tỉnh/ Thành phố',
    'region': "Khu vực",
    'admission_code': 'Mã xét tuyển',
    'university_code': 'Mã Trường',
    'university_name': 'Tên Trường',
    'major_code': 'Mã Ngành',
    'major_name': 'Tên Ngành',
    'subject_group': 'Tổ hợp',
    'point': 'Điểm chuẩn',
    'note': 'Ghi chú',
    'year': 'Năm',
    'institution_type': 'Loại đơn vị'
}

# Rename columns for each DataFrame in the dictionary
for name, df in dfs.items():
    df.rename(columns=column_mapping, inplace=True)

full_df.rename(columns=column_mapping, inplace=True)
full_df.head()

,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Loại đơn vị
0,TVU,Đại Học Trà Vinh,7210201,Âm nhạc học,N00,15.00,NaN,2018,Trà Vinh,Miền Nam,DVT,Công lập
1,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.90,Tốt nghiệp THPT,2018,Hồ Chí Minh,Miền Nam,SPK,Công lập
2,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90, XDHB",27.25,Học bạ,2018,Hồ Chí Minh,Miền Nam,SPK,Công lập
3,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,DGNLHCM,22.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2018,Hồ Chí Minh,Miền Nam,SPK,Công lập
4,VNUHCM - UIT,UIT - Đại Học Công Nghệ Thông Tin – Đại Học Qu...,7480202_CLCA,An toàn thông tin,"A00, A01, D01",20.10,chất lượng cao,2018,Hồ Chí Minh,Miền Nam,QSC,Công lập


In [48]:
full_df = full_df[[ 'Tỉnh/ Thành phố', 'Khu vực', 'Mã xét tuyển','Mã Trường', 'Tên Trường','Mã Ngành', 'Tên Ngành', 'Tổ hợp', 'Điểm chuẩn', 'Ghi chú', 'Năm', 'Loại đơn vị']]

In [49]:
full_df

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
0,Trà Vinh,Miền Nam,DVT,TVU,Đại Học Trà Vinh,7210201,Âm nhạc học,N00,15.00,NaN,2018,Công lập
1,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.90,Tốt nghiệp THPT,2018,Công lập
2,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90, XDHB",27.25,Học bạ,2018,Công lập
3,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,DGNLHCM,22.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2018,Công lập
4,Hồ Chí Minh,Miền Nam,QSC,VNUHCM - UIT,UIT - Đại Học Công Nghệ Thông Tin – Đại Học Qu...,7480202_CLCA,An toàn thông tin,"A00, A01, D01",20.10,chất lượng cao,2018,Công lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37413,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập
37414,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập
37415,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập
37416,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập


### **Khám phá dữ liệu**

#### 1. Mỗi dòng có ý nghĩa gì?

In [50]:
full_df.sample(5)

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
4674,Hồ Chí Minh,Miền Nam,QSX,VNUHCM-USSH,USSH - Đại Học Khoa Học Xã Hội và Nhân Văn – Đ...,7580112_BT,Đô thị học,"A01, D01, C00, D14",18.20,Phân hiệu Bến Tre,2019,Công lập
21416,Hồ Chí Minh,Miền Nam,QSB,VNUHCM-UT,UT - Đại Học Bách Khoa – Đại Học Quốc Gia TPHCM,219,Công nghệ thực phẩm,"A00, B00, D07",25.70,CT chất lượng caoĐiểm thi TN THPT,2021,Công lập
32079,Hà Nội,Miền Bắc,TLA,TLU,Đại Học Thủy Lợi (Cơ sở 1),TLA106,Công nghệ thông tin,"A00, A01, D01, D07",25.89,Tốt nghiệp THPT; Điểm Toán >=8.6; TTNV<=7,2023,Công lập
22757,Quảng Nam,Miền Trung - Tây Nguyên,DQU,QNU,Đại Học Quảng Nam,7229010,Lịch sử,"C00, D14, C19",13.00,Điểm thi TN THPT,2022,Công lập
3474,Hồ Chí Minh,Miền Nam,SPS,HCMUE,Đại Học Sư Phạm TPHCM,7140218,Sư phạm Lịch sử,"C00, D14",19.75,NaN,2018,Công lập


Mỗi dòng là thông tin về điểm chuẩn của một ngành, tại một trường, vào một năm cụ thể.

##### Kiểm tra trùng lặp

In [51]:
if len(full_df[full_df.duplicated()]) == 0:
    print(f'Không có dòng nào trùng lặp trong dữ liệu.')
else:
    print(f'Tồn tại {len(full_df[full_df.duplicated()])} dòng bị trùng lặp trong dữ liệu.')

Tồn tại 15 dòng bị trùng lặp trong dữ liệu.


Xuất các dòng trùng lặp ra file csv (Để coi chơi thôi chứ cũng drop à :v)

In [52]:
full_df[full_df.duplicated()].to_csv('./data/dup.csv',  encoding='utf-8-sig')

Loại bỏ các dòng trùng

In [53]:
full_df.drop_duplicates(inplace=True)

C:\Users\letra\AppData\Local\Temp\ipykernel_41308\2742331558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.drop_duplicates(inplace=True)


Kiểm tra lại

In [54]:
if len(full_df[full_df.duplicated()]) == 0:
    print(f'Không có dòng nào trùng lặp trong dữ liệu.')
else:
    print(f'Tồn tại {len(full_df[full_df.duplicated()])} dòng bị trùng lặp trong dữ liệu.')

Không có dòng nào trùng lặp trong dữ liệu.


#### 2. Mỗi cột có ý nghĩa gì?

Các cột và thông tin tương ứng:
- `Mã Trường`: Tên viết tắt của trường (Không phải mã trên hệ thống của bộ).

- `Tên Trường`: Tên của Trường đại học.

- `	Mã Ngành`: Mã ngành tuyển sinh.

- `	Tên Ngành`: Tên ngành tuyển sinh.

- `	Tổ hợp`: Các tổ hợp môn xét tuyển.

- `	Điểm chuẩn`: Điểm chuẩn tương ứng với tổ hợp.

- `	Ghi chú`: Ghi chú thêm (thường là thông tin về điều kiện bổ sung, hoặc phương thức xét tuyển).

- `	Năm`: Năm tuyển sinh.

#### 3. Mỗi cột hiện có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp cần xử lý tiếp hay không?

In [55]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37403 entries, 0 to 37417
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Tỉnh/ Thành phố  37403 non-null  object 
 1   Khu vực          37403 non-null  object 
 2   Mã xét tuyển     37403 non-null  object 
 3   Mã Trường        37403 non-null  object 
 4   Tên Trường       37403 non-null  object 
 5   Mã Ngành         37403 non-null  object 
 6   Tên Ngành        37403 non-null  object 
 7   Tổ hợp           37380 non-null  object 
 8   Điểm chuẩn       37403 non-null  float64
 9   Ghi chú          25072 non-null  object 
 10  Năm              37403 non-null  int64  
 11  Loại đơn vị      37403 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 3.7+ MB


Các cột có kiểu dữ liệu phù hợp.

#### 4. Với mỗi cột, các giá trị (dạng số, dạng phân loại) được phân bố như thế nào?

##### 4.1. Các cột dạng số

In [56]:
numerical_columns = full_df.select_dtypes(include=['int64', 'float64']).columns
print("Numerical columns:", numerical_columns)

def calculate_missing_ratio(df):
    # Tính tỷ lệ dữ liệu thiếu cho từng cột
    missing_ratios = df.isna().mean() * 100

    # Trả về Series chứa tỷ lệ dữ liệu thiếu cho từng cột
    return missing_ratios

# Gọi hàm tính missing ratio và in kết quả
missing_ratios = calculate_missing_ratio(full_df[numerical_columns])
print('Missing ratio:')
print(missing_ratios)
full_df.describe()

Numerical columns: Index(['Điểm chuẩn', 'Năm'], dtype='object')
Missing ratio:
Điểm chuẩn    0.0
Năm           0.0
dtype: float64


,Điểm chuẩn,Năm
count,37403.000000,37403.000000
mean,74.307980,2020.857418
std,183.171632,1.678067
min,0.000000,2018.000000
25%,16.000000,2019.000000
50%,20.000000,2021.000000
75%,25.100000,2022.000000
max,1035.000000,2023.000000


In [57]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Năm' hoặc cột 'Điểm chuẩn' (nếu có)
missing_rows = full_df[full_df['Năm'].isna() | full_df['Điểm chuẩn'].isna()]
missing_rows

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị


In [58]:
full_df.drop(missing_rows.index, inplace=True)
full_df.reset_index(drop=True, inplace=True)

C:\Users\letra\AppData\Local\Temp\ipykernel_41308\3106419298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.drop(missing_rows.index, inplace=True)


In [59]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Năm' hoặc cột 'Điểm chuẩn' (nếu có)
missing_rows = full_df[full_df['Năm'].isna() | full_df['Điểm chuẩn'].isna()]
missing_rows

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị


Thấy min của điểm chuẩn là 0 (kỳ lạ) nên ta sẽ xử lý các dòng có điểm chuẩn = 0 một chút.


In [60]:
full_df[full_df['Điểm chuẩn']==0]

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
61,Hà Nội,Miền Bắc,LNH,VNUF,Đại Học Lâm Nghiệp ( Cơ sở 1 ),7620112,Bảo vệ thực vật,"A00, B00, D01, A16",0.0,NaN,2018,Công lập
103,Hà Nội,Miền Bắc,LNH,VNUF,Đại Học Lâm Nghiệp ( Cơ sở 1 ),7620105,Chăn nuôi,"A00, B00, D08, A16",0.0,NaN,2018,Công lập
160,Hà Nội,Miền Bắc,BKA,HUST,Đại Học Bách Khoa Hà Nội,ME-LUH,Cơ điện tử,"A00, A01, D07",0.0,ĐH Leibniz Hannover (Đức),2018,Công lập
184,Hà Nội,Miền Bắc,LNH,VNUF,Đại Học Lâm Nghiệp ( Cơ sở 1 ),7549001A,Công nghệ chế biến lâm sản,"A00, D01, D07, A16",0.0,Chất lượng cao,2018,Công lập
188,Hà Nội,Miền Bắc,LNH,VNUF,Đại Học Lâm Nghiệp ( Cơ sở 1 ),7549001,Công nghệ chế biến lâm sản,"A00, D01, D07, A16",0.0,NaN,2018,Công lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37040,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520212,Kỹ thuật y sinh,"A00, B00, A01, A02, XDHB",0.0,NaN,2023,Dân lập
37042,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520320,Kỹ thuật môi trường,"A00, B00, A02, C08, XDHB",0.0,NaN,2023,Dân lập
37044,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7720802,Quản trị bệnh viện,"B00, B03, C01, C02, XDHB",0.0,NaN,2023,Dân lập
37046,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7810202,Quản trị nhà hàng và dịch vụ ăn uống,"A00, A01, D01, C00, XDHB",0.0,NaN,2023,Dân lập


Ta thấy có rất nhiều ngành có điểm chuẩn =0, tuy nhiên, một số sẽ có chú thích riêng. Vậy, đối với các ngành có điểm chuẩn =0 và không có chú thích thì mình xem như năm đó không có tuyển sinh. 
> => Mình loại bỏ các dòng này.

In [61]:
diemchuan_0 = full_df[(full_df['Điểm chuẩn']==0) & (full_df['Ghi chú'].isna())]
diemchuan_0

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
61,Hà Nội,Miền Bắc,LNH,VNUF,Đại Học Lâm Nghiệp ( Cơ sở 1 ),7620112,Bảo vệ thực vật,"A00, B00, D01, A16",0.0,NaN,2018,Công lập
103,Hà Nội,Miền Bắc,LNH,VNUF,Đại Học Lâm Nghiệp ( Cơ sở 1 ),7620105,Chăn nuôi,"A00, B00, D08, A16",0.0,NaN,2018,Công lập
188,Hà Nội,Miền Bắc,LNH,VNUF,Đại Học Lâm Nghiệp ( Cơ sở 1 ),7549001,Công nghệ chế biến lâm sản,"A00, D01, D07, A16",0.0,NaN,2018,Công lập
228,Hà Nội,Miền Bắc,BKA,HUST,Đại Học Bách Khoa Hà Nội,ED2,Công nghệ giáo dục,"A00, A01, D01",0.0,NaN,2018,Công lập
261,Hà Nội,Miền Bắc,LNH,VNUF,Đại Học Lâm Nghiệp ( Cơ sở 1 ),7510203,Công nghệ kỹ thuật cơ điện tử,"A00, A01, D01, A16",0.0,NaN,2018,Công lập
...,...,...,...,...,...,...,...,...,...,...,...,...
37040,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520212,Kỹ thuật y sinh,"A00, B00, A01, A02, XDHB",0.0,NaN,2023,Dân lập
37042,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7520320,Kỹ thuật môi trường,"A00, B00, A02, C08, XDHB",0.0,NaN,2023,Dân lập
37044,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7720802,Quản trị bệnh viện,"B00, B03, C01, C02, XDHB",0.0,NaN,2023,Dân lập
37046,Cần Thơ,Miền Nam,DNC,NCTU,Đại học Nam Cần Thơ,7810202,Quản trị nhà hàng và dịch vụ ăn uống,"A00, A01, D01, C00, XDHB",0.0,NaN,2023,Dân lập


In [62]:
shape = full_df.shape

# Drop các dòng có trong diemchuan_0
full_df.drop(diemchuan_0.index, inplace=True)

# Reset lại chỉ số sau khi drop
full_df.reset_index(drop=True, inplace=True)

print(f"Dữ liệu trước xử lý có {shape[0]} dòng và {shape[1]} cột.")
print(f"Dữ liệu   sau xử lý có {full_df.shape[0]} dòng và {full_df.shape[1]} cột.")

Dữ liệu trước xử lý có 37403 dòng và 12 cột.
Dữ liệu   sau xử lý có 36736 dòng và 12 cột.


C:\Users\letra\AppData\Local\Temp\ipykernel_41308\1664910539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.drop(diemchuan_0.index, inplace=True)


##### 4.2. Các cột dạng phân loại

In [63]:
categorical_columns = full_df.select_dtypes(include=['object']).columns
categorical_columns

Index(['Tỉnh/ Thành phố', 'Khu vực', 'Mã xét tuyển', 'Mã Trường', 'Tên Trường',
       'Mã Ngành', 'Tên Ngành', 'Tổ hợp', 'Ghi chú', 'Loại đơn vị'],
      dtype='object')

In [64]:
def calculate_column_profile(column):
    missing_ratio = len(column[column.isna()]) * 100 / len(column)
    num_diff_vals = len(column.dropna().unique())
    diff_vals = column.dropna().unique()
    return pd.Series([missing_ratio, num_diff_vals, diff_vals], index=['missing_ratio', 'num_diff_vals', 'diff_vals'])

cate_col_profiles_df = full_df[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Ghi chú,Loại đơn vị
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013611,31.750871,0.0
num_diff_vals,52,3,277,256,289,2598,677,2492,5265,2
diff_vals,"[Trà Vinh, Hồ Chí Minh, Hà Nội, Thái Nguyên, Đ...","[Miền Nam, Miền Bắc, Miền Trung - Tây Nguyên]","[DVT, SPK, QSC, DKC, BVS, BVH, KMA, DCT, DDT, ...","[TVU, HCMUTE, VNUHCM - UIT, HUTECH, PTIT, ACT,...","[Đại Học Trà Vinh, Đại Học Sư Phạm Kỹ Thuật TP...","[7210201, 7480202D, 7480202_CLCA, 7480202, 772...","[Âm nhạc học, An toàn thông tin, Bác sĩ Răng -...","[N00, A00, A01, D01, D90, A00, A01, D01, D90, ...","[Tốt nghiệp THPT, Học bạ, Đánh giá năng lực Đạ...","[Công lập, Dân lập]"


(Lại) Thấy `Tổ hợp` có missing, trong khi dữ liệu này đáng lẽ không nên miss.  Mình in ra xem thử

In [65]:
# Tìm và in ra các dòng có dữ liệu thiếu trong cột 'Mã Ngành' hoặc cột 'Tổ hợp' (nếu có)
missing_rows = full_df[full_df['Tổ hợp'].isna()]
missing_rows

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị
1331,Hà Nội,Miền Bắc,TDD,TDU,Đại học Thành Đô,6810103,Hướng dẫn du lịch,NaN,0.0,Tốt nghiệp THPT,2018,Dân lập
3023,Hà Nội,Miền Bắc,TDD,TDU,Đại học Thành Đô,6810201,Quản trị khách sạn,NaN,0.0,Tốt nghiệp THPT,2018,Dân lập
3922,Hà Nội,Miền Bắc,TDD,TDU,Đại học Thành Đô,6220206,Tiếng Anh,NaN,0.0,Tốt nghiệp THPT,2018,Dân lập
3927,Hà Nội,Miền Bắc,TDD,TDU,Đại học Thành Đô,6220213,Tiếng Nhật,NaN,0.0,Tốt nghiệp THPT,2018,Dân lập
4864,Hà Nội,Miền Bắc,HVN,VNUA,Học Viện Nông Nghiệp Việt Nam,7140215,Sư phạm Kỹ thuật nông nghiệp,NaN,0.0,Tuyển thẳng đối với thí sinh đạt học lực giỏi ...,2019,Công lập


Thấy toàn bộ missing là tuyển thẳng, nên Replace "Tuyển thẳng"

In [66]:
full_df.loc[full_df['Tổ hợp'].isna(), 'Tổ hợp'] = 'Tuyển thẳng'

In [67]:
cate_col_profiles_df = full_df[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Ghi chú,Loại đơn vị
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.750871,0.0
num_diff_vals,52,3,277,256,289,2598,677,2493,5265,2
diff_vals,"[Trà Vinh, Hồ Chí Minh, Hà Nội, Thái Nguyên, Đ...","[Miền Nam, Miền Bắc, Miền Trung - Tây Nguyên]","[DVT, SPK, QSC, DKC, BVS, BVH, KMA, DCT, DDT, ...","[TVU, HCMUTE, VNUHCM - UIT, HUTECH, PTIT, ACT,...","[Đại Học Trà Vinh, Đại Học Sư Phạm Kỹ Thuật TP...","[7210201, 7480202D, 7480202_CLCA, 7480202, 772...","[Âm nhạc học, An toàn thông tin, Bác sĩ Răng -...","[N00, A00, A01, D01, D90, A00, A01, D01, D90, ...","[Tốt nghiệp THPT, Học bạ, Đánh giá năng lực Đạ...","[Công lập, Dân lập]"


In [68]:
full_df[full_df['Tỉnh/ Thành phố'].isna()].to_csv('test.csv',  encoding='utf-8-sig', index=False)

### **Thêm nhóm ngành**

In [69]:
nhom_nganh_df = pd.read_csv('./data/university/nhom_nganh.csv')

# Tạo từ điển tra cứu từ nhom_nganh_df và đảm bảo các khóa đều là chuỗi
nhom_nganh_dict = {str(k): v for k, v in zip(nhom_nganh_df['Mã nhóm ngành'], nhom_nganh_df['Nhóm ngành'])}

# Hàm để lấy Mã nhóm ngành và Nhóm ngành dựa trên Mã Ngành
def get_nhom_nganh(ma_nganh):
    if pd.isna(ma_nganh):
        return pd.Series([None, None])
    ma_nhom_nganh = ma_nganh[:3]  # Trích xuất 3 ký tự đầu
    if ma_nhom_nganh in nhom_nganh_dict:
        nhom_nganh = nhom_nganh_dict[ma_nhom_nganh]
        return pd.Series([ma_nhom_nganh, nhom_nganh])
    else:
        return pd.Series([None, None])

# Áp dụng hàm để thêm cột Mã nhóm ngành và Nhóm ngành vào full_df
full_df[['Mã nhóm ngành', 'Nhóm ngành']] = full_df['Mã Ngành'].apply(get_nhom_nganh)
full_df

C:\Users\letra\AppData\Local\Temp\ipykernel_41308\1221631991.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df[['Mã nhóm ngành', 'Nhóm ngành']] = full_df['Mã Ngành'].apply(get_nhom_nganh)
C:\Users\letra\AppData\Local\Temp\ipykernel_41308\1221631991.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df[['Mã nhóm ngành', 'Nhóm ngành']] = full_df['Mã Ngành'].apply(get_nhom_nganh)


,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,Mã nhóm ngành,Nhóm ngành
0,Trà Vinh,Miền Nam,DVT,TVU,Đại Học Trà Vinh,7210201,Âm nhạc học,N00,15.00,NaN,2018,Công lập,721,Nghệ thuật
1,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.90,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin
2,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90, XDHB",27.25,Học bạ,2018,Công lập,748,Máy tính và công nghệ thông tin
3,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,DGNLHCM,22.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2018,Công lập,748,Máy tính và công nghệ thông tin
4,Hồ Chí Minh,Miền Nam,QSC,VNUHCM - UIT,UIT - Đại Học Công Nghệ Thông Tin – Đại Học Qu...,7480202_CLCA,An toàn thông tin,"A00, A01, D01",20.10,chất lượng cao,2018,Công lập,748,Máy tính và công nghệ thông tin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36731,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn
36732,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,731,Khoa học xã hội và hành vi
36733,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn
36734,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.00,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn


### **Tách cột Tổ hợp**

In [70]:
# Tách cột "Tổ hợp" theo dấu phẩy và tạo các dòng mới
#df_expanded = full_df.assign(subject_group=full_df['Tổ hợp'].str.split(', ')).explode('Tổ hợp')
df_expanded = full_df.assign(subject_group=full_df['Tổ hợp'].str.split(', ')).explode('subject_group')

df_expanded.reset_index(drop=True, inplace=True)
df_expanded

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,Mã nhóm ngành,Nhóm ngành,subject_group
0,Trà Vinh,Miền Nam,DVT,TVU,Đại Học Trà Vinh,7210201,Âm nhạc học,N00,15.0,NaN,2018,Công lập,721,Nghệ thuật,N00
1,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,A00
2,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,A01
3,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,D01
4,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,D90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114329,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn,DGNLHCM
114330,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,731,Khoa học xã hội và hành vi,DGNLHCM
114331,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn,DGNLHCM
114332,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn,DGNLHCM


In [71]:
condition2 = (df_expanded['Ghi chú'].str.contains('ao đẳng', na=False))
df_expanded.drop(df_expanded[condition2].index, inplace=True)

condition2 = (df_expanded['Ghi chú'].str.contains('CAO ĐẲNG', na=False))
df_expanded.drop(df_expanded[condition2].index, inplace=True)

# Lọc và xóa các dòng có subject_group là 'XDHB' và 'ọc bạ' tồn tại trong Ghi chú
condition = (df_expanded['subject_group'] == 'XDHB') & (df_expanded['Ghi chú'].str.contains('Học bạ', na=False))|(df_expanded['subject_group'] == 'XDHB') & (df_expanded['Ghi chú'].str.contains('HỌC BẠ', na=False))
df_expanded.drop(df_expanded[condition].index, inplace=True)


condition3 = (df_expanded['subject_group'] == 'XDHB') & (df_expanded['Tổ hợp'].str.contains(', XDHB', na=False))
df_expanded.drop(df_expanded[condition3].index, inplace=True)

# Đặt lại chỉ mục của full_df và lưu thay đổi này vào DataFrame gốc
df_expanded.reset_index(drop=True, inplace=True)
df_expanded



,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,Mã nhóm ngành,Nhóm ngành,subject_group
0,Trà Vinh,Miền Nam,DVT,TVU,Đại Học Trà Vinh,7210201,Âm nhạc học,N00,15.0,NaN,2018,Công lập,721,Nghệ thuật,N00
1,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,A00
2,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,A01
3,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,D01
4,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,D90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107441,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn,DGNLHCM
107442,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,731,Khoa học xã hội và hành vi,DGNLHCM
107443,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn,DGNLHCM
107444,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn,DGNLHCM


In [72]:
categorical_columns = df_expanded.select_dtypes(include=['object']).columns
cate_col_profiles_df = df_expanded[categorical_columns].agg(calculate_column_profile)
cate_col_profiles_df

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Ghi chú,Loại đơn vị,Mã nhóm ngành,Nhóm ngành,subject_group
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.30782,0.0,10.942241,10.942241,0.0
num_diff_vals,52,3,277,256,289,2582,676,2486,5167,2,24,24,247
diff_vals,"[Trà Vinh, Hồ Chí Minh, Hà Nội, Thái Nguyên, Đ...","[Miền Nam, Miền Bắc, Miền Trung - Tây Nguyên]","[DVT, SPK, QSC, DKC, BVS, BVH, KMA, DCT, DDT, ...","[TVU, HCMUTE, VNUHCM - UIT, HUTECH, PTIT, ACT,...","[Đại Học Trà Vinh, Đại Học Sư Phạm Kỹ Thuật TP...","[7210201, 7480202D, 7480202_CLCA, 7480202, 772...","[Âm nhạc học, An toàn thông tin, Bác sĩ Răng -...","[N00, A00, A01, D01, D90, A00, A01, D01, D90, ...","[Tốt nghiệp THPT, Học bạ, Đánh giá năng lực Đạ...","[Công lập, Dân lập]","[721, 748, 772, 744, 732, 734, 785, 762, 786, ...","[Nghệ thuật, Máy tính và công nghệ thông tin, ...","[N00, A00, A01, D01, D90, DGNLHCM, C01, C02, B..."


In [73]:
# df_expanded.to_csv("./data/preprocessing/full.csv",  encoding='utf-8-sig', index=False)

In [74]:
# Tạo cột 'Loại điểm' dựa trên 'Ghi chú' và 'Tổ hợp'
df_expanded['Loại điểm'] = ''
df_expanded.loc[(df_expanded['Ghi chú']=='Xét học bạ')|(df_expanded['Ghi chú'].str.contains('Học bạ', na=False))|(df_expanded['Tổ hợp'].str.contains('XDHB', na=False)), 'Loại điểm'] = 'Học bạ'

# Cập nhật 'Loại điểm' dựa trên 'Tổ hợp'
df_expanded.loc[((df_expanded['Tổ hợp'].str.contains('DGNL', na=False)) | (df_expanded['Tổ hợp'].str.contains('DGTD', na=False))), 'Loại điểm'] = 'Kỳ thi riêng'
df_expanded.loc[df_expanded['Loại điểm'] == '', 'Loại điểm'] = 'THPTQG'

df_expanded

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã Ngành,Tên Ngành,Tổ hợp,Điểm chuẩn,Ghi chú,Năm,Loại đơn vị,Mã nhóm ngành,Nhóm ngành,subject_group,Loại điểm
0,Trà Vinh,Miền Nam,DVT,TVU,Đại Học Trà Vinh,7210201,Âm nhạc học,N00,15.0,NaN,2018,Công lập,721,Nghệ thuật,N00,THPTQG
1,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,A00,THPTQG
2,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,A01,THPTQG
3,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,D01,THPTQG
4,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,7480202D,An toàn thông tin,"A00, A01, D01, D90",26.9,Tốt nghiệp THPT,2018,Công lập,748,Máy tính và công nghệ thông tin,D90,THPTQG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107441,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220209,Ngôn ngữ Nhật,DGNLHCM,761.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn,DGNLHCM,Kỳ thi riêng
107442,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7310601,Quốc tế học,DGNLHCM,648.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,731,Khoa học xã hội và hành vi,DGNLHCM,Kỳ thi riêng
107443,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220203,Ngôn ngữ Pháp,DGNLHCM,643.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn,DGNLHCM,Kỳ thi riêng
107444,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,7220202,Ngôn ngữ Nga,DGNLHCM,613.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023,Công lập,722,Nhân văn,DGNLHCM,Kỳ thi riêng


In [75]:
df_expanded.drop(columns = ['Tổ hợp'], inplace=True)
df_expanded.rename(columns={'subject_group': 'Tổ hợp'}, inplace=True)
df_expanded = df_expanded[['Tỉnh/ Thành phố', 'Khu vực', 'Mã xét tuyển','Mã Trường', 'Tên Trường', 'Mã nhóm ngành', 'Nhóm ngành', 'Mã Ngành', 'Tên Ngành', 'Loại điểm', 'Tổ hợp', 'Điểm chuẩn', 'Ghi chú', 'Năm']]
df_expanded 

,Tỉnh/ Thành phố,Khu vực,Mã xét tuyển,Mã Trường,Tên Trường,Mã nhóm ngành,Nhóm ngành,Mã Ngành,Tên Ngành,Loại điểm,Tổ hợp,Điểm chuẩn,Ghi chú,Năm
0,Trà Vinh,Miền Nam,DVT,TVU,Đại Học Trà Vinh,721,Nghệ thuật,7210201,Âm nhạc học,THPTQG,N00,15.0,NaN,2018
1,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,748,Máy tính và công nghệ thông tin,7480202D,An toàn thông tin,THPTQG,A00,26.9,Tốt nghiệp THPT,2018
2,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,748,Máy tính và công nghệ thông tin,7480202D,An toàn thông tin,THPTQG,A01,26.9,Tốt nghiệp THPT,2018
3,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,748,Máy tính và công nghệ thông tin,7480202D,An toàn thông tin,THPTQG,D01,26.9,Tốt nghiệp THPT,2018
4,Hồ Chí Minh,Miền Nam,SPK,HCMUTE,Đại Học Sư Phạm Kỹ Thuật TPHCM,748,Máy tính và công nghệ thông tin,7480202D,An toàn thông tin,THPTQG,D90,26.9,Tốt nghiệp THPT,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107441,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,722,Nhân văn,7220209,Ngôn ngữ Nhật,Kỳ thi riêng,DGNLHCM,761.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023
107442,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,731,Khoa học xã hội và hành vi,7310601,Quốc tế học,Kỳ thi riêng,DGNLHCM,648.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023
107443,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,722,Nhân văn,7220203,Ngôn ngữ Pháp,Kỳ thi riêng,DGNLHCM,643.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023
107444,Đà Nẵng,Miền Trung - Tây Nguyên,DDF,UFLS,Đại Học Ngoại Ngữ – Đại Học Đà Nẵng,722,Nhân văn,7220202,Ngôn ngữ Nga,Kỳ thi riêng,DGNLHCM,613.0,Đánh giá năng lực Đại học Quốc gia TPHCM,2023


In [76]:
df_expanded.to_csv("./data/diemchuan_full.csv",  encoding='utf-8-sig', index=False)